In [1]:
import pandas as pd
import xarray as xr
import geopandas as gpd
import rasterio
import numpy as np
import richdem as rd
from rasterio.transform import rowcol
from datetime import datetime

# Load and prepare VIIRS data
gdf = gpd.read_file('custom_date_fires.geojson')

# Convert VIIRS data to DEM's CRS (EPSG:4269)
with rasterio.open('USGS3DEP_30m_33.5_34.5_-119.0_-118.0.tif') as src:
    dem_crs = src.crs
gdf = gdf.to_crs(dem_crs)

# Load and prepare ERA5 data
era5_wind = xr.open_dataset('era5_wind_la.nc', engine='netcdf4').rename({'valid_time': 'time'})

# Load DEM and calculate slope
with rasterio.open('USGS3DEP_30m_33.5_34.5_-119.0_-118.0.tif') as src:
    dem = src.read(1)
    transform = src.transform
    dem_bounds = src.bounds
    dem_height, dem_width = dem.shape

# Clip VIIRS points to DEM bounds
gdf_clipped = gdf.cx[
    dem_bounds.left:dem_bounds.right,
    dem_bounds.bottom:dem_bounds.top
]

# Calculate slope using richdem
dem_rd = rd.rdarray(dem, no_data=-9999)
slope = rd.TerrainAttribute(dem_rd, attrib='slope_degrees')

def get_wind_at_point(lat, lon, viirs_time):
    """Get wind components at specific location and time"""
    # Convert VIIRS time to numpy datetime
    era5_time = np.datetime64(viirs_time)
    
    return era5_wind.sel(
        latitude=lat,
        longitude=lon,
        time=era5_time,
        method='nearest'
    )

# Process features with error handling
features = []
for _, row in gdf_clipped.iterrows():
    try:
        # Get wind data
        wind = get_wind_at_point(row.geometry.y, row.geometry.x, row['acq_date'])
        
        # Convert coordinates to DEM grid indices
        x, y = row.geometry.x, row.geometry.y
        row_idx, col_idx = rowcol(transform, x, y)
        
        # Check array bounds
        if (0 <= row_idx < dem_height) and (0 <= col_idx < dem_width):
            elevation = dem[row_idx, col_idx]
            slope_value = slope[row_idx, col_idx]
        else:
            print(f"Point out of DEM bounds: ({x}, {y})")
            elevation = np.nan
            slope_value = np.nan
        
        features.append({
            'latitude': row.geometry.y,
            'longitude': row.geometry.x,
            'frp': row['frp'],
            'u10': wind.u10.item(),
            'v10': wind.v10.item(),
            'elevation': elevation,
            'slope': slope_value,
            'confidence': row['confidence']
        })
        
    except Exception as e:
        print(f"Skipping row due to error: {str(e)}")
        continue

features_df = pd.DataFrame(features)
print("Successfully processed features:")
print(features_df.head())
print(f"\nTotal features: {len(features_df)}")
print(f"NaN values:\n{features_df.isna().sum()}")


Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)



A Slope calculation (degrees)
C Horn, B.K.P., 1981. Hill shading and the reflectance map. Proceedings of the IEEE 69, 14–47. doi:10.1109/PROC.1981.11918

t Wall-time = 0.234428======================= ] (99% - 0.0s - 1 threads)


Successfully processed features:
   latitude  longitude   frp       u10       v10   elevation      slope  \
0  33.81573 -118.23775  1.05  1.259857  3.371613    1.781858  69.389099   
1  34.15556 -118.19301  1.56  0.662201  0.849152  434.277283  80.075356   
2  34.29366 -118.80275  1.33  0.376068  0.480011  297.056061  56.920059   
3  34.33566 -118.51929  1.25  0.539154  0.854034  581.089966  73.065117   
4  34.42990 -118.64459  0.86  0.599701 -1.086395  350.062622  75.363266   

  confidence  
0          n  
1          n  
2          n  
3          n  
4          n  

Total features: 1731
NaN values:
latitude      0
longitude     0
frp           0
u10           0
v10           0
elevation     0
slope         0
confidence    0
dtype: int64
